In [10]:
import gym
import pygame
import random
import matplotlib.pyplot as plt
import pylab as pl

from numpy import array, pi, cos, sin
from env.map.RVO import RVO_update, Agent
from env.map.grid import OccupancyGridMap
from env.mav.drone import Drone2D
from env.planner.primitive import Primitive, Trajectory2D
from env.planner.yaw_planner import LookAhead, Oxford, NoControl
from IPython import display


from env.map.utils import *
from env.config import *      

state_machine = {
        'WAIT_FOR_GOAL':0,
        'GOAL_REACHED' :1,
        'PLANNING'     :2,
        'EXECUTING'    :3
    }
class Drone2DEnv(gym.Env):
     
    def __init__(self):
        
        self.dt = 1/10
        
        self.obstacles = {
            'circular_obstacles'  : [[320, 240, 50]],
            'rectangle_obstacles' : [[100, 100, 100, 40], [400, 300, 100, 30]]
        }
        
        # Define workspace model for RVO model (approximate using circles)
        self.ws_model = obs_dict_to_ws_model(self.obstacles)
        
        # Setup pygame environment
        pygame.init()
        self.dim = MAP_SIZE
        self.screen = pygame.display.set_mode(self.dim)
        self.clock = pygame.time.Clock()
        
        # Define physical setup
        self.agents = []
        if ENABLE_DYNAMIC:
            i = 1
            while(i <= N_AGENTS):
                theta = 2 * pi * i / N_AGENTS
                x = array((cos(theta), sin(theta))) #+ random.uniform(-1, 1)
                vel = -x * PEDESTRIAN_MAX_SPEED
                pos = (random.uniform(200, 440), random.uniform(120, 360))
                new_agent = Agent(pos, (0., 0.), PEDESTRIAN_RADIUS, PEDESTRIAN_MAX_SPEED, vel, self.dt)
                if check_collision(self.agents, new_agent, self.ws_model):
                    self.agents.append(new_agent)
                    i += 1

        self.map_gt = OccupancyGridMap(MAP_GRID_SCALE, self.dim, 2)
        self.map_gt.init_obstacles(self.obstacles, self.agents)
    
        self.drone = Drone2D(self.dim[0] / 2, DRONE_RADIUS + self.map_gt.x_scale, -90, self.dt, self.dim)
        self.planner = Primitive(self.drone)

        self.target_list = [np.array([520, 100]), np.array([120, 50]), np.array([120, 380]), np.array([520, 380])]
        
        self.trajectory = Trajectory2D()
        self.state = state_machine['WAIT_FOR_GOAL']
        self.state_changed = False
        self.replan_count = 0

        # Define action and observation space
        self.observation = {
            'drone':self.drone,
            'trajectory':self.trajectory
        }
    
    def step(self, a):

        self.state_changed = False
        # Update state machine
        if self.state == state_machine['GOAL_REACHED']:
            self.state = state_machine['WAIT_FOR_GOAL']
            self.state_changed = True
        # Update gridmap for dynamic obstacles
        self.map_gt.update_dynamic_grid(self.agents)

        # Raycast module
        self.drone.raycasting(self.map_gt, self.agents)

        # Update moving agent position
        if ENABLE_DYNAMIC:
            RVO_update(self.agents, self.ws_model)
            for agent in self.agents:
                agent.step(self.map_gt.x_scale, self.map_gt.y_scale, self.dim[0], self.dim[1],  self.dt)
        
        # Set target point
        if self.state == state_machine['WAIT_FOR_GOAL']:
            self.planner.set_target(self.target_list[-1])
            self.target_list.pop()
            self.state = state_machine['PLANNING']
        # mouse = pygame.mouse.get_pressed()
        # if mouse[0]:
        #     success = False
        #     x, y = pygame.mouse.get_pos()
        #     self.planner.set_target(np.array([x, y]))
        #     self.trajectory, success = self.planner.plan(np.array([self.drone.x, self.drone.y]), self.drone.velocity, self.drone.map, self.agents, self.dt)
        #     self.state_changed = True
        #     if not success:
        #         self.drone.brake()
        #         self.state = state_machine['PLANNING']
        #     else:
        #         self.state = state_machine['EXECUTING']

        #Plan
        if self.state == state_machine['PLANNING']:
            self.trajectory, success = self.planner.plan(np.array([self.drone.x, self.drone.y]), self.drone.velocity, self.drone.map, self.agents, self.dt)
            if not success:
                self.drone.brake()
                # print("path not found, replanning")
            else:
                # print("path found")
                self.state_changed = True
                self.state = state_machine['EXECUTING']

        # If collision detected for planned trajectory, replan
        swep_map = np.zeros_like(self.map_gt.grid_map)
        for i, pos in enumerate(self.trajectory.positions):
            swep_map[int(pos[0]//MAP_GRID_SCALE), int(pos[1]//MAP_GRID_SCALE)] = i * self.dt
            for agent in self.agents:
                if agent.seen:
                    estimate_pos = agent.estimate_pos + i * self.dt * agent.estimate_vel
                    if norm(estimate_pos - pos) <= self.drone.radius + agent.radius:
                        self.state = state_machine['PLANNING']
                        self.state_changed = True   
        obs_map = np.where((self.drone.map.grid_map==0) | (self.drone.map.grid_map==2), 0, 1)
        if np.sum(obs_map * swep_map) > 0:
            self.state = state_machine['PLANNING']
            self.state_changed = True

        # Execute trajectory
        if self.trajectory.positions != [] :
            self.drone.velocity = self.trajectory.velocities[0]
            self.drone.x = round(self.trajectory.positions[0][0])
            self.drone.y = round(self.trajectory.positions[0][1])
            self.trajectory.pop()
            if self.trajectory.positions == []:
                self.state_changed = True
                self.state = state_machine['GOAL_REACHED']
        
        # Execute gaze control
        self.drone.step_yaw(a)
        
        # Print state machine
        # if self.state_changed:
        #     if self.state == state_machine['GOAL_REACHED']:
        #         print("state: goal reached")
        #     elif self.state == state_machine['WAIT_FOR_GOAL']:
        #         print("state: wait for goal")
        #     elif self.state == state_machine['PLANNING']:
        #         print("state: planning")
        #     elif self.state == state_machine['EXECUTING']:
        #         print("state: executing trajectory")

        # wrap up observation
        self.observation = {
            'drone':self.drone,
            'trajectory':self.trajectory
        }

        # Return reward
        if self.drone.is_collide(self.map_gt, self.agents):
            reward = -100
            done = True
        elif self.state == state_machine['GOAL_REACHED']:
            reward = 100
            done = False
            if len(self.target_list) == 0:
                done = True
        else:
            reward = -1
            done = False

        return self.observation, reward, done
    
    def reset(self):
        self.__init__()
        
    def render(self, mode='human'):
        # keys = pygame.key.get_pressed()
        # if keys[pygame.K_LEFT]:
        #     self.drone.yaw += 2
        # if keys[pygame.K_RIGHT]:
        #     self.drone.yaw -= 2
        # pygame.event.pump() # process event queue
        
        # self.map_gt.render(self.screen, color_dict)
        self.drone.map.render(self.screen, color_dict)
        self.drone.render(self.screen)
        for ray in self.drone.rays:
            pygame.draw.line(
                self.screen,
                (100,100,100),
                (self.drone.x, self.drone.y),
                ((ray['coords'][0]), (ray['coords'][1]))
        )
        # draw_static_obstacle(self.screen, self.obstacles, (200, 200, 200))
        
        if len(self.trajectory.positions) > 1:
            pygame.draw.lines(self.screen, (100,100,100), False, self.trajectory.positions)

        if ENABLE_DYNAMIC:
            for agent in self.agents:
                agent.render(self.screen)
        
        fps = round(self.clock.get_fps())
        if (fps >= 40):
            fps_color = (0,102,0)
        elif(fps >= 20):
            fps_color = (255, 153, 0)
        else:
            fps_color = (204, 0, 0)
        default_font = pygame.font.SysFont('Arial', 15)
        pygame.Surface.blit(self.screen,
            default_font.render('FPS: '+str(fps), False, fps_color),
            (0, 0)
        )
        
        pygame.display.update()
        self.clock.tick(60)
    


In [11]:
from tqdm import tqdm
env = Drone2DEnv()
# policy = LookAhead(env.dt)
policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

    env.render()

        # window = max(int(i / 50), 10)

        # if i % 50 == 0:
        #     colors = ['b', 'g', 'r']
        #     fig = plt.gcf()
        #     fig.set_size_inches(16, 4)
        #     plt.clf()
        #     pl.subplot(1, 1, 1)
        #     pl.xlabel('environment steps')
        #     pl.ylabel('episode return')

        #     pl.plot(steps, rewards, colors[0])
        #     display.clear_output(wait=True)
        #     display.display(plt.gcf())

  0%|          | 0/10000 [00:00<?, ?it/s]/home/smoggy/thesis/gym-Drone2D-ActivePerception/planner/yaw_planner.py:71: RuntimeWarning: invalid value encountered in divide
  view_map = np.where(np.arccos(((x - drone.x)*vec_yaw[0] + (y - drone.y)*vec_yaw[1]) / np.sqrt((drone.x - x)**2 + (drone.y - y)**2)) <= view_angle, np.where(((drone.x - x)**2 + (drone.y - y)**2 <= drone.yaw_depth ** 2), 1, 0), 0)
 10%|▉         | 959/10000 [00:28<04:31, 33.36it/s]


KeyboardInterrupt: 

In [5]:
print(success / (success + fail))

0.873015873015873


In [6]:
env = Drone2DEnv()
policy = LookAhead(env.dt)
# policy = Oxford(env.dt, env.dim)
# plt.ion()
max_step = 10000
rewards = []
steps = []
success = 0
fail = 0
for i in tqdm(range(max_step)):
    
    a = policy.plan(env.observation)
    observation, reward, done= env.step(a)
    if reward == 100:
        success += 1
    if reward == -100:
        fail += 1
    if done:
        env.reset()

    rewards.append(reward)
    steps.append(i)

 16%|█▋        | 1632/10000 [00:18<01:35, 87.92it/s] 

collision with dynamic obstacles


 21%|██▏       | 2148/10000 [00:25<01:44, 75.29it/s] 

collision with dynamic obstacles


 34%|███▍      | 3435/10000 [00:40<01:25, 77.05it/s] 

...

 80%|████████  | 8028/10000 [01:32<00:16, 117.55it/s]

collision with dynamic obstacles


100%|█████████▉| 9979/10000 [01:55<00:00, 130.33it/s]

collision with dynamic obstacles


100%|██████████| 10000/10000 [01:55<00:00, 86.52it/s]


In [7]:
print(success / (success + fail))

0.8983050847457628
